## Advanced options for multioutput handling

### Imports

In [1]:
import os
# set the device to run
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

os.makedirs('../data', exist_ok=True)

import numpy as np
import pandas as pd
from pandas import Series, DataFrame

from sklearn.model_selection import train_test_split

from py_boost import GradientBoosting, SketchBoost

# strategies to deal with multiple outputs
from py_boost.multioutput.sketching import *
from py_boost.multioutput.target_splitter import *

### Downloading data from OpenML

In [2]:
!wget https://www.openml.org/data/get_csv/19335692/file1c556677f875.csv -O ../data/helena.csv

--2024-06-03 11:16:12--  https://www.openml.org/data/get_csv/19335692/file1c556677f875.csv
Resolving www.openml.org (www.openml.org)... 131.155.11.11
Connecting to www.openml.org (www.openml.org)|131.155.11.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘../data/helena.csv’

../data/helena.csv      [      <=>           ]  14.56M  11.1MB/s    in 1.3s    

2024-06-03 11:16:38 (11.1 MB/s) - ‘../data/helena.csv’ saved [15271704]



In [3]:
data = pd.read_csv('../data/helena.csv')

data

,class,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27
0,41,0.005521,0.080556,0.110417,0.490822,0.586406,0.066414,0.092206,0.116352,0.379310,...,-0.342986,78.6894,17.237800,21.504200,14.43730,17.378000,9.61674,-0.609370,1.044830,1.481790
1,92,0.049398,0.147917,0.541667,0.542865,0.515608,0.105128,0.475550,0.049555,0.383460,...,2.639370,59.7879,5.393410,3.819610,11.49240,3.929470,5.91423,1.409210,4.749540,1.103820
2,24,0.548663,1.000000,1.000000,0.397029,0.627398,1.023440,1.004220,0.027381,0.451337,...,0.137427,58.1429,-3.365980,-0.037489,10.63470,2.660180,3.93377,-0.898220,2.137790,1.054470
3,29,0.023073,0.206250,0.238889,0.622998,0.764067,0.202599,0.177892,0.071232,0.531712,...,0.477009,55.4798,-1.051090,-4.755360,13.36710,2.852060,9.65162,0.224397,-0.220216,-0.273287
4,91,0.224427,0.433333,0.902083,0.814199,0.576879,0.344413,0.822975,0.026121,0.425875,...,0.521306,76.8475,-19.371700,32.270700,9.41442,4.343450,8.67710,-1.587580,1.117870,-0.545338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65191,88,0.007292,0.152083,0.061111,0.114431,0.406104,0.143170,0.053086,0.129365,0.215442,...,1.265300,53.2951,-1.416430,2.173900,13.66950,1.588520,2.02855,0.619052,0.622377,-0.363035
65192,77,0.411279,1.000000,0.430556,0.503805,0.207163,1.003740,0.412067,0.017673,0.044771,...,-2.842380,91.1178,-0.009207,-2.224830,1.30504,0.898489,1.80362,-2.726140,-0.184329,-0.476441
65193,71,0.999352,1.000000,1.000000,0.501360,0.501075,0.999384,0.999414,0.009636,0.000648,...,0.213472,84.4141,2.042450,13.849800,7.24428,1.443890,4.00495,-0.749115,1.025130,0.096257
65194,24,0.206175,0.383333,0.944444,0.749915,0.550936,0.292477,0.830921,0.033542,0.430515,...,0.879472,61.4110,17.354200,5.566660,16.22600,10.049400,6.04195,0.400956,0.375599,0.644575


In [4]:
data['class'].value_counts()

class
78    4005
55    3063
40    2992
39    2623
38    2216
      ... 
56     121
75     121
32     119
34     116
10     111
Name: count, Length: 100, dtype: int64

Looks like we have 100 classes here ...

In [5]:
X = data.drop('class', axis=1).values.astype('float32')
y = data['class'].values.astype('int32')

X, X_test, y, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

### Traininig a multiclass model

A simple use case for training a multiclass problem is the same as for regression. By default py_boost builds multioutout trees to handle multioutput problems (single tree outputs a vector of length 100 for 100 class task).


In [6]:
%%time
model = GradientBoosting(
    'crossentropy',
     ntrees=10000, lr=0.03, verbose=100, es=300, lambda_l2=1, gd_steps=1,
     subsample=1, colsample=1, min_data_in_leaf=10, use_hess=True, 
     max_bin=256, max_depth=6, debug=True
)

model.fit(X, y, 
          eval_sets = [{'X': X_test, 'y': y_test}])

[11:16:41] Stdout logging level is INFO.
[11:16:41] GDBT train starts. Max iter 10000, early stopping rounds 300
[11:16:42] Iter 0; Sample 0, Crossentropy = 4.285501528878882; 
[11:16:48] Iter 100; Sample 0, Crossentropy = 2.776514677504898; 
[11:16:55] Iter 200; Sample 0, Crossentropy = 2.653074406126989; 
[11:17:01] Iter 300; Sample 0, Crossentropy = 2.6105114034455075; 
[11:17:09] Iter 400; Sample 0, Crossentropy = 2.5903803264512146; 
[11:17:16] Iter 500; Sample 0, Crossentropy = 2.578669364783949; 
[11:17:24] Iter 600; Sample 0, Crossentropy = 2.5710490146934406; 
[11:17:31] Iter 700; Sample 0, Crossentropy = 2.5672467579364637; 
[11:17:39] Iter 800; Sample 0, Crossentropy = 2.5645683355765847; 
[11:17:47] Iter 900; Sample 0, Crossentropy = 2.562009018108624; 
[11:17:54] Iter 1000; Sample 0, Crossentropy = 2.5607421238591717; 
[11:18:02] Iter 1100; Sample 0, Crossentropy = 2.560525910660894; 
[11:18:09] Iter 1200; Sample 0, Crossentropy = 2.5599909857168726; 
[11:18:17] Iter 1300;

In [7]:
%%time
pred = model.predict(X_test)
pred.shape

CPU times: user 2.62 s, sys: 932 ms, total: 3.56 s
Wall time: 3.56 s


(13040, 100)

### Sketching strategies to speedup training

Computational costs of training multioutput models drastically increase when number of output grows. We implemented a few strategies to simplify tree structure search via gradinet matrix sketching:

* ***RandomSamplingSketch*** (recommended for use_hess=True)
* ***RandomProjectionSketch*** (recommended for use_hess=False)
* ***TopOutputsSketch***
* ***SVDSketch*** (needs RAPIDS (cuml) to be installed)

To understand how the sketch strategy works you can explore our [paper](https://papers.neurips.cc/paper_files/paper/2022/hash/a36c3dbe676fa8445715a31a90c66ab3-Abstract-Conference.html)

Let us check, how it works.

In [8]:
%%time

sketch = RandomProjectionSketch(1)
# sketch = RandomSamplingSketch(10)
# sketch = TopOutputsSketch(10)
# sketch = SVDSketch(n_components=1)

model = GradientBoosting(
    'crossentropy',
    ntrees=10000, lr=0.03, verbose=100, es=300, lambda_l2=1, gd_steps=1, 
    subsample=1, colsample=1, min_data_in_leaf=10, use_hess=False, 
    max_bin=256, max_depth=6,
    multioutput_sketch=sketch, debug=True
)

model.fit(X, y, eval_sets = [{'X': X_test, 'y': y_test}])

[11:18:41] Stdout logging level is INFO.
[11:18:41] GDBT train starts. Max iter 10000, early stopping rounds 300
[11:18:42] Iter 0; Sample 0, Crossentropy = 4.3778085843527705; 
[11:18:43] Iter 100; Sample 0, Crossentropy = 2.8830529720409874; 
[11:18:44] Iter 200; Sample 0, Crossentropy = 2.734289345289923; 
[11:18:45] Iter 300; Sample 0, Crossentropy = 2.6735559168503396; 
[11:18:46] Iter 400; Sample 0, Crossentropy = 2.6399241611790485; 
[11:18:47] Iter 500; Sample 0, Crossentropy = 2.616061705571971; 
[11:18:48] Iter 600; Sample 0, Crossentropy = 2.59947891765521; 
[11:18:50] Iter 700; Sample 0, Crossentropy = 2.5870792166676453; 
[11:18:51] Iter 800; Sample 0, Crossentropy = 2.576737247042549; 
[11:18:52] Iter 900; Sample 0, Crossentropy = 2.5697475582952003; 
[11:18:53] Iter 1000; Sample 0, Crossentropy = 2.5641300738272945; 
[11:18:54] Iter 1100; Sample 0, Crossentropy = 2.558705242032253; 
[11:18:55] Iter 1200; Sample 0, Crossentropy = 2.5550109417491056; 
[11:18:56] Iter 1300;

### SketchBoost

Alternatively you can use SketchBoost class with built in sketching strateges. Just define number of outputs to keep

In [11]:
%%time
model = SketchBoost(
    'crossentropy',
     ntrees=10000, lr=0.03, verbose=100, es=300, lambda_l2=1, gd_steps=1, 
     subsample=1, colsample=1, min_data_in_leaf=10, 
     max_bin=256, max_depth=6,
)

model.fit(X, y, eval_sets = [{'X': X_test, 'y': y_test}])

[11:19:11] Stdout logging level is INFO.
[11:19:11] GDBT train starts. Max iter 10000, early stopping rounds 300
[11:19:11] Iter 0; Sample 0, Crossentropy = 4.377808584599463; 
[11:19:12] Iter 100; Sample 0, Crossentropy = 2.8830498432815292; 
[11:19:13] Iter 200; Sample 0, Crossentropy = 2.7342919171166464; 
[11:19:14] Iter 300; Sample 0, Crossentropy = 2.6735435580216023; 
[11:19:15] Iter 400; Sample 0, Crossentropy = 2.6399076860148907; 
[11:19:16] Iter 500; Sample 0, Crossentropy = 2.6160517786375967; 
[11:19:17] Iter 600; Sample 0, Crossentropy = 2.5994730816804905; 
[11:19:19] Iter 700; Sample 0, Crossentropy = 2.587071393747626; 
[11:19:20] Iter 800; Sample 0, Crossentropy = 2.5767342482806783; 
[11:19:21] Iter 900; Sample 0, Crossentropy = 2.5697525339702767; 
[11:19:22] Iter 1000; Sample 0, Crossentropy = 2.564136382620364; 
[11:19:23] Iter 1100; Sample 0, Crossentropy = 2.5587091536295925; 
[11:19:24] Iter 1200; Sample 0, Crossentropy = 2.555019138642021; 
[11:19:25] Iter 130

In [12]:
%%time
pred = model.predict(X_test)
pred.shape

CPU times: user 948 ms, sys: 20 ms, total: 968 ms
Wall time: 963 ms


(13040, 100)

#### We see a nice speed up and sometimes even a better accuracy!

#### These modifications allow us to train a model faster than CatBoost does (with a similar setup).

In [13]:
%%time
from catboost import CatBoostClassifier

model = CatBoostClassifier(
                       grow_policy='Depthwise', 
                       bootstrap_type='Bernoulli',
                       subsample=1.,
                       border_count=256, 
                       iterations=10000, 
                       od_wait=300,
                       max_depth=6, 
                       devices='0:0', 
                       learning_rate=0.03, 
                       l2_leaf_reg=1, 
                       min_data_in_leaf=10, 
                       score_function='L2',
                       model_shrink_mode='Constant',
                       **{'task_type': 'GPU', 'verbose': 100, }
                    )

model.fit(X, y, eval_set = (X_test, y_test))    

0:	learn: 4.2940511	test: 4.3045901	best: 4.3045901 (0)	total: 38.3ms	remaining: 6m 22s
100:	learn: 2.7026070	test: 2.8639822	best: 2.8639822 (100)	total: 3.4s	remaining: 5m 33s
200:	learn: 2.4526124	test: 2.7287556	best: 2.7287556 (200)	total: 6.57s	remaining: 5m 20s
300:	learn: 2.3213712	test: 2.6789284	best: 2.6789284 (300)	total: 9.56s	remaining: 5m 8s
400:	learn: 2.2232473	test: 2.6516266	best: 2.6516266 (400)	total: 12.5s	remaining: 4m 58s
500:	learn: 2.1449564	test: 2.6351182	best: 2.6351182 (500)	total: 15.3s	remaining: 4m 50s
600:	learn: 2.0773091	test: 2.6237368	best: 2.6237368 (600)	total: 18.1s	remaining: 4m 43s
700:	learn: 2.0146058	test: 2.6149345	best: 2.6149318 (699)	total: 20.9s	remaining: 4m 36s
800:	learn: 1.9531135	test: 2.6072940	best: 2.6072841 (799)	total: 23.7s	remaining: 4m 31s
900:	learn: 1.8969737	test: 2.6016194	best: 2.6016194 (900)	total: 26.5s	remaining: 4m 27s
1000:	learn: 1.8410815	test: 2.5963373	best: 2.5963373 (1000)	total: 29.3s	remaining: 4m 23s
11